# HOWTO: gourdian in python

In [1]:
import logging
logging.basicConfig(level=logging.INFO, format='%(message)s')

import pandas as pd

from gourdian import gtypes, GourdianClient

## I. Quickstart

Use any `pandas.DataFrame` to fetch a relevant gourdian dataset, given only the mapping from your DataFrame's columns to our special `gtypes`:

In [2]:
# 1. Load your project's data into a "local" dataframe.
local_df = pd.read_csv('~/Motor_Vehicle_Collisions_-_Crashes.csv', dtype={'ZIP CODE': str})

# Perform whatever cleanup you need as usual.
local_df = local_df[(local_df['LATITUDE'] != 0) & (~local_df['LATITUDE'].isna())]
local_df['CRASH DATE'] = pd.to_datetime(local_df['CRASH DATE'])
local_df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2013-07-26,8:00,NaN,NaN,40.822555,-73.885714,"(40.8225547, -73.8857144)",NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,2881336,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,NaN,NaN,NaN
3,2021-03-21,19:45,BROOKLYN,11208,40.671352,-73.881820,"(40.671352, -73.88182)",NaN,NaN,1000 SUTTER AVENUE,...,Unspecified,NaN,NaN,NaN,4400695,Bike,Sedan,NaN,NaN,NaN
12,2021-03-20,4:30,BROOKLYN,11233,40.677444,-73.917340,"(40.677444, -73.91734)",NaN,NaN,12A LOUIS PLACE,...,Unspecified,NaN,NaN,NaN,4400479,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
16,2021-03-19,12:39,BROOKLYN,11235,40.585747,-73.953735,"(40.585747, -73.953735)",NaN,NaN,1501 VOORHIES AVENUE,...,NaN,NaN,NaN,NaN,4400100,Sedan,NaN,NaN,NaN,NaN
17,2015-04-08,18:00,MANHATTAN,10128,40.778806,-73.947966,"(40.7788056, -73.9479659)",EAST 89 STREET,1 AVENUE,NaN,...,NaN,NaN,NaN,NaN,3199792,VAN,VAN,NaN,NaN,NaN


In [3]:
# 2. Join your local dataframe to a "remote" gourdian dataset of your choosing.
REMOTE_DATASET = 'eric/noaa_gsod.global_summary_of_day'

# Specify a mapping between your local dataframe columns and the remote gourdian dataset.
match = local_df.gourdian.match(endpointer=REMOTE_DATASET, how={
    'LATITUDE': gtypes.Point.Latitude,
    'LONGITUDE': gtypes.Point.Longitude,
})
match.describe()

# While you can get stats and data paginators from match, we'll just fetch everything as one big dataframe.
remote_df = match.df()
remote_df

# LayoutMatch
Endpointer: `eric/noaa_gsod.global_summary_of_day@lat_lng`

## Stats
Matched: 783K rows (across 10 chunks)
Filesize: 18.9MB (146MB uncompressed)

## Matched Label Columns (2)
- lat: Point.Latitude(step=0.703125, head=-90.0)
- lng: Point.Longitude(step=1.40625, head=-180.0)

## Chunks (10)
- chunk|+030.234375|-090.000000.csv.gz (73806 rows)
- chunk|+034.453125|-087.187500.csv.gz (51113 rows)
- chunk|+040.078125|-075.937500.csv.gz (125631 rows)
- chunk|+040.078125|-074.531250.csv.gz (202167 rows)
- chunk|+040.781250|-074.531250.csv.gz (131204 rows)
- chunk|+041.484375|-080.156250.csv.gz (18635 rows)
- chunk|+041.484375|-077.343750.csv.gz (21046 rows)
- chunk|+042.187500|-080.156250.csv.gz (55822 rows)
- chunk|+042.187500|-074.531250.csv.gz (58261 rows)
- chunk|+042.890625|-077.343750.csv.gz (45515 rows)


,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,74768613820,1980-01-01,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",47.4,24,34.4,24,...,9.9,999.9,60.1,*,39.9,,0.00,D,999.9,0
1,74768613820,1980-01-02,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",48.4,24,36.9,24,...,4.9,999.9,62.1,*,39.9,,0.00,D,999.9,100000
2,74768613820,1980-01-03,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",55.8,24,48.1,24,...,8.9,999.9,66.0,*,48.9,,0.00,D,999.9,100000
3,74768613820,1980-01-04,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",49.9,24,43.0,24,...,15.0,28.0,65.8,,41.9,,0.01,G,999.9,110000
4,74768613820,1980-01-05,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",40.9,24,24.9,24,...,14.0,999.9,54.0,,32.9,,0.00,G,999.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45510,72514699999,2003-12-27,43.35000,-76.38300,145.0,"OSWEGO CO, NY US",32.4,24,26.8,24,...,18.1,21.0,39.0,,23.0,,0.02,G,999.9,0
45511,72514699999,2003-12-28,43.35000,-76.38300,145.0,"OSWEGO CO, NY US",30.2,24,22.4,24,...,7.0,999.9,45.0,,21.0,,0.00,G,999.9,0
45512,72514699999,2003-12-29,43.35000,-76.38300,145.0,"OSWEGO CO, NY US",39.2,24,23.8,24,...,6.0,999.9,50.0,,21.0,,0.00,I,999.9,0
45513,72514699999,2003-12-30,43.35000,-76.38300,145.0,"OSWEGO CO, NY US",39.5,24,33.3,24,...,32.1,41.0,50.0,,30.9,,0.30,G,999.9,11000


In [4]:
# 3. We've fetched only the relevant parts of the remote dataset. Check our work.
local_df['LATITUDE'].min(), local_df['LATITUDE'].max()

(30.78418, 43.344444)

In [5]:
remote_df['LATITUDE'].min(), remote_df['LATITUDE'].max()

(30.283, 43.47)

## II. Tutorial: Fetch data relevant to your work

In this short tutorial we will be using a _local_ DataFrame, read from a CSV file already on your computer.

This local DataFrame represents the dataset for the problem you are actually working on; for this tutorial, you are a researcher studying road and vehicle safety in New York!  

> A _local_ DataFrame is data you already have, which can be augmented by gourdian datasets.

Our local data will be the "Motor Vehicle Collisions - Crashes" dataset published by NYC OpenData, which can be downloaded manually from https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95.


### A. Load your data into a _local_ pandas DataFrame like usual

It is typical to read your own raw data and then clean it up by removing missing values (converting data types, etc.)  This step is specific to your local data, and has to be done manually and by inspection:

In [6]:
# Local dataframe: sourced from NYC; export as CSV from the buttons on the top right.
# -> https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95
local_df = pd.read_csv('~/Motor_Vehicle_Collisions_-_Crashes.csv', dtype={'ZIP CODE': str})

# Remove rows with unknown LATITUDE and LONGITUDE.
local_df = local_df[~local_df['LATITUDE'].isna()]
local_df = local_df[~local_df['LONGITUDE'].isna()]
# In this dataset, we know that 0.0 also implies "unknown" for LATITUDE and LONGITUDE.
local_df = local_df[local_df['LATITUDE'] != 0]
local_df = local_df[local_df['LONGITUDE'] != 0]
# Remove crazy values for LATITUDE and LONGITUDE.
local_df = local_df[(local_df['LATITUDE'] >= -90) & (local_df['LATITUDE'] <= 90)]
local_df = local_df[(local_df['LONGITUDE'] >= -180) & (local_df['LATITUDE'] <= 180)]
# We also know the 'CRASH DATE' column can be made into a proper datetime.
local_df['CRASH DATE'] = pd.to_datetime(local_df['CRASH DATE'])

local_df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2013-07-26,8:00,NaN,NaN,40.822555,-73.885714,"(40.8225547, -73.8857144)",NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,2881336,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,NaN,NaN,NaN
3,2021-03-21,19:45,BROOKLYN,11208,40.671352,-73.881820,"(40.671352, -73.88182)",NaN,NaN,1000 SUTTER AVENUE,...,Unspecified,NaN,NaN,NaN,4400695,Bike,Sedan,NaN,NaN,NaN
12,2021-03-20,4:30,BROOKLYN,11233,40.677444,-73.917340,"(40.677444, -73.91734)",NaN,NaN,12A LOUIS PLACE,...,Unspecified,NaN,NaN,NaN,4400479,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
16,2021-03-19,12:39,BROOKLYN,11235,40.585747,-73.953735,"(40.585747, -73.953735)",NaN,NaN,1501 VOORHIES AVENUE,...,NaN,NaN,NaN,NaN,4400100,Sedan,NaN,NaN,NaN,NaN
17,2015-04-08,18:00,MANHATTAN,10128,40.778806,-73.947966,"(40.7788056, -73.9479659)",EAST 89 STREET,1 AVENUE,NaN,...,NaN,NaN,NaN,NaN,3199792,VAN,VAN,NaN,NaN,NaN


### B. Fetch a _remote_ dataset relevant to your problem

Today we are interested in understanding the relationship between NYC car accidents and the weather.

Luckily for us, daily weather data is published by the National Oceanic and Atmospheric Administration (NOAA) in the United States.  This _remote_ dataset contains rows covering the entire world, going all the way back to the 1930s.

> A remote dataset is hosted by gourdian and may be published by any person or organization.

A gourdian dataset is nothing more than a collection of rows allocated to one or more tables, and some common metadata and licensing information.  Each table has its own table schema, and every row in the same table is guaranteed to have the same shape.

As we can see below, this dataset published by NOAA contains just one table called `global_summary_of_day`.  It contains ~156 million rows at the time of this writing:

In [7]:
client = GourdianClient()

client.dataset('eric/noaa_gsod').describe()

# Global Surface Summary of Day :: US National Oceanic and Atmospheric Administration (NOAA)
Endpointer: `eric/noaa_gsod`

The Global Surface Summary of the Day is derived from The Integrated Surface Hourly (ISH) dataset. The ISH dataset includes global data obtained from the USAF Climatology Center, located in the Federal Climate Complex with NCDC.

The latest daily summary data are normally available 1-2 days after the date-time of the observations used in the daily summaries.  The online data files begin with 1929 and are at the time of this writing at the Version 8 software level. Over 9000 stations' data are typically available.

The daily elements included in the dataset (as available from each station) are:
- Mean temperature (.1 Fahrenheit)
- Mean dew point (.1 Fahrenheit)
- Mean sea level pressure (.1 mb)
- Mean station pressure (.1 mb)
- Mean visibility (.1 miles)
- Mean wind speed (.1 knots)
- Maximum sustained wind speed (.1 knots)
- Maximum wind gust (.1 knots)
- Maximum t

In most cases, you will only be interested in a small subset of a remote dataset.  Our local "Motor Vehicle Collisions - Crashes" DataFrame, for example, only concerns a slice of the globe and contains no rows before 2012:

In [8]:
local_df['LATITUDE'].min(), local_df['LATITUDE'].max()

(30.78418, 43.344444)

In [9]:
local_df['LONGITUDE'].min(), local_df['LONGITUDE'].max()

(-89.13527, -32.768513)

In [10]:
local_df['CRASH DATE'].min(), local_df['CRASH DATE'].max()

(Timestamp('2012-07-01 00:00:00'), Timestamp('2021-03-21 00:00:00'))

#### 1. Summarizing a local dataframe as a Query

Fetching the entire remote `global_summary_of_day` table from the NOAA dataset would be a waste for our work today, as it contains millions of unrelated rows about places and dates we aren't interested in.

To pluck out only the relevant pieces of a remote dataset, the gourdian python library first creates a Query object which contains ranges of values that summarize your local DataFrame.

> A _Query_ summarizes a local DataFrame into the parts of a remote dataset you are interested in.

This summarization step allows you to download relevant pieces of remote datasets without having to provide any of your actual data to gourdian, keeping it private on your computer. Only the summary contained in a Query is ever needed.

While a Query may be created by hand, usually we generate one automatically from a local DataFrame:

In [11]:
query = local_df.gourdian.query(how={
    'CRASH DATE': gtypes.Datetime.Year,
    'LATITUDE': gtypes.Point.Latitude,
    'LONGITUDE': gtypes.Point.Longitude,
})
query.describe()

# Query (18 subqueries)
- OR
  + Datetime.Year
    - [2012.0 ... 2016.0] (len=4.0)
  + Point.Latitude
    - [40.4296875 ... 40.95703125] (len=0.52734375)
  + Point.Longitude
    - [-74.53125 ... -73.4765625] (len=1.0546875)
- OR
  + Datetime.Year
    - [2016.0 ... 2017.0] (len=1.0)
  + Point.Latitude
    - [30.76171875 ... 30.9375] (len=0.17578125)
  + Point.Longitude
    - [-89.296875 ... -88.9453125] (len=0.3515625)
- OR
  + Datetime.Year
    - [2016.0 ... 2017.0] (len=1.0)
  + Point.Latitude
    - [34.62890625 ... 34.8046875] (len=0.17578125)
  + Point.Longitude
    - [-86.8359375 ... -86.484375] (len=0.3515625)
- OR
  + Datetime.Year
    - [2016.0 ... 2017.0] (len=1.0)
  + Point.Latitude
    - [40.4296875 ... 40.78125] (len=0.3515625)
  + Point.Longitude
    - [-74.53125 ... -73.4765625] (len=1.0546875)
- OR
  + Datetime.Year
    - [2016.0 ... 2017.0] (len=1.0)
  + Point.Latitude
    - [40.60546875 ... 40.78125] (len=0.17578125)
  + Point.Longitude
    - [-33.046875 ... -32.6953125

To create this Query, columns in our local "Motor Vehicle Collisions - Crashes" DataFrame are mapped to columns on the remote dataset, allowing like values to be compared.

While this mapping can be done in many ways, it is usually sufficient to specify the _gtypes_ of our local columns (as we have done above), which are gourdian-specific data types for things like locations and dates.

> A local column's _gtype_ encodes its gourdian-specific type, which determines the remote columns it will be compared to

As you can see above, our local DataFrame has been summarized into a Query that can be used to fetch pieces of a remote dataset that lie within relevant ranges of latitudes, longitudes, and years (corresponding to the gtypes `Point.Latitude`, `Point.Longitude`  and `Datetime.Year`).

#### 2. Matching to a remote dataset and fetching relevant pieces of it

To actually fetch rows from the remote NOAA dataset, we simply match our Query against a specific remote dataset table and ask for a DataFrame in return.

Since the NOAA dataset has just one table called `global_summary_of_day`, we'll use that one:

In [12]:
# Summarize our local dataframe into a Query.
query = local_df.gourdian.query(how={
    'CRASH DATE': gtypes.Datetime.Year,
    'LATITUDE': gtypes.Point.Latitude,
    'LONGITUDE': gtypes.Point.Longitude,
})

# Get the Table containing the remote data we want.
table = client.table('eric/noaa_gsod.global_summary_of_day')

# Explicitly match our query against that table, and ask for a dataframe via df().
match = query.match_table(table=table)
remote_df = match.df()
remote_df.head()

GET 'https://data2.gourdian.net/files/datasets/eric/noaa_gsod/global_summary_of_day/lat_lng/chunks/chunk|+040.781250|-075.937500.csv.gz'


,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,74768613820,1980-01-01,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",47.4,24,34.4,24,...,9.9,999.9,60.1,*,39.9,,0.00,D,999.9,0
1,74768613820,1980-01-02,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",48.4,24,36.9,24,...,4.9,999.9,62.1,*,39.9,,0.00,D,999.9,100000
2,74768613820,1980-01-03,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",55.8,24,48.1,24,...,8.9,999.9,66.0,*,48.9,,0.00,D,999.9,100000
3,74768613820,1980-01-04,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",49.9,24,43.0,24,...,15.0,28.0,65.8,,41.9,,0.01,G,999.9,110000
4,74768613820,1980-01-05,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",40.9,24,24.9,24,...,14.0,999.9,54.0,,32.9,,0.00,G,999.9,0


#### 3. Generating a Query and fetching pieces of a remote dataset in one step

We have just shown how to explicitly summarize a local DataFrame into a Query, and then how to fetch remote data using that Query. When debugging, this process can be useful to understand.

However, it is usually easier to perform these 2 steps at the same time:

In [13]:
match = local_df.gourdian.match(
    endpointer='eric/noaa_gsod.global_summary_of_day',
    how={
    'CRASH DATE': gtypes.Datetime.Year,
    'LATITUDE': gtypes.Point.Latitude,
    'LONGITUDE': gtypes.Point.Longitude,
    },
)
match.df().head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,74768613820,1980-01-01,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",47.4,24,34.4,24,...,9.9,999.9,60.1,*,39.9,,0.00,D,999.9,0
1,74768613820,1980-01-02,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",48.4,24,36.9,24,...,4.9,999.9,62.1,*,39.9,,0.00,D,999.9,100000
2,74768613820,1980-01-03,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",55.8,24,48.1,24,...,8.9,999.9,66.0,*,48.9,,0.00,D,999.9,100000
3,74768613820,1980-01-04,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",49.9,24,43.0,24,...,15.0,28.0,65.8,,41.9,,0.01,G,999.9,110000
4,74768613820,1980-01-05,30.41667,-88.91667,10.1,"BILOXI KEESLER AFB, MS US",40.9,24,24.9,24,...,14.0,999.9,54.0,,32.9,,0.00,G,999.9,0


### C. Getting more details about remote table matching

#### 1. Layouts

Just as a gourdian dataset is organized into one or more tables, each table is itself organized into one or more _layouts_.

However, unlike tables (which have distinct schemas and their own rows), each layout contains a full copy of its table's rows sorted and split in a different way.

> A _layout_ contains a full copy of a table's rows split into multiple files by some useful criteria, which can be downloaded independently.

A single table typically has several layouts defined on it. One may contain _chunks_ of rows split up by latitude and longitude (the `lat_lng` layout), and another may split the rows instead by year and month (the `year_month` layout). Both layouts contain 100% of a table's rows; they differ only in how those rows are sorted and split.

> A _chunk_ contains related rows belonging to a single layout, and is the smallest unit of a dataset.

Queries primarily concerned with certain geographic regions may fetch data from the `lat_lng` layout, for example, while Queries about a specific timeframe are better served by `year_month`.  With only these 2 layouts defined, however, Queries about both a specific place and a specific time will be forced to choose one or the other.  This often results in irrelevant rows being retrieved!

Currently, layouts are designed by hand to make it possible for a Query to fetch only the relevant pieces of a table, while keeping the number of irrelevant rows downloaded to a minimum.

Since layouts determine the ways in which a Query can selectively download pieces of a table, they are very important! If no pre-built layout exists on a table for your Query, the gourdian library has no choice but to download the entire thing.

The layouts available on a table are shown at the end of `table.describe()`:

In [14]:
table.describe()

# Global Surface Summary of Day
Endpointer: `eric/noaa_gsod.global_summary_of_day`

# Global Surface Summary of Day Data (GSOD)
## National Centers for Environmental Information

OVER 9000 WORLDWIDE STATIONS

10/28/2020

## Special Notes

The data summaries provided here are based on data exchanged under the World Meteorological Organization (WMO) World Weather Watch Program according to WMO Resolution 40 (Cg-XII). This allows WMO member countries to place restrictions on the use or re-export of their data for commercial purposes outside of the receiving country.  Data for selected countries may, at times, not be available through this system.

Those countries' data summaries and products which are available here are intended for free and unrestricted use in research, education, and other non-commercial activities.  However, for non-U.S. locations' data, the data or any derived product shall not be provided to other users or be used for the re-export of commercial services.

To determi

A table's layouts can also be programtically inspected:

In [15]:
table.layouts

(<Layout 'eric/noaa_gsod.global_summary_of_day@lat_lng' label_columns=['lat', 'lng']>,
 <Layout 'eric/noaa_gsod.global_summary_of_day@lat_lng_year' label_columns=['lat', 'lng', 'year']>,
 <Layout 'eric/noaa_gsod.global_summary_of_day@year_month' label_columns=['year', 'month']>)

#### 2. LayoutMatches

While a table may have multiple layouts, only a single layout will be used to fetch data for a specific Query.  

The specifics of matching a Query to a given layout can be retrieved from a _LayoutMatch_ object. These details are used by the gourdian client library to identify the best layout to use when fetching rows for every Query, and you might find them interesting too.

> A _LayoutMatch_ represents how a specific layout will be used to satisfy a given Query.

We've already seen how to get a LayoutMatch from a Query, but here it is again:

In [16]:
match = local_df.gourdian.match(
    endpointer='eric/noaa_gsod.global_summary_of_day',
    how={
    'CRASH DATE': gtypes.Datetime.Year,
    'LATITUDE': gtypes.Point.Latitude,
    'LONGITUDE': gtypes.Point.Longitude,
    },
)
match

<LayoutMatch 'eric/noaa_gsod.global_summary_of_day@lat_lng' chunks=11 rows=838K csv_bytes=157M gz_bytes=20.3M>

In [17]:
match.fetch_all_stats()

{'matched_num_chunks': 11,
 'matched_num_rows': 838306,
 'matched_csv_bytes': 157491184,
 'matched_gz_bytes': 20250337}

In [18]:
# A LayoutMatch also includes a pointer to the specific layout that was used.
match.layout

<Layout 'eric/noaa_gsod.global_summary_of_day@lat_lng' label_columns=['lat', 'lng']>

#### 3. Generating all LayoutMatches for a table

Sometimes fetching data from a remote dataset can return some surprising rows which are irrelevant to your Query. This is usually because a layout perfectly designed for your Query does not exist, forcing the gourdian client to pick the best from what is available. 

Because the rows returned by a Query are determined by the layout that was chosen, it can be helpful to see all the possible matches that were considered and discarded:

In [19]:
query = local_df.gourdian.query(
    endpointer='eric/noaa_gsod.global_summary_of_day',
    how={
    'CRASH DATE': gtypes.Datetime.Year,
    'LATITUDE': gtypes.Point.Latitude,
    'LONGITUDE': gtypes.Point.Longitude,
    },
)
matches = list(query.all_layout_matches(layouts=table.layouts))
matches

[<LayoutMatch 'eric/noaa_gsod.global_summary_of_day@lat_lng' chunks=11 rows=838K csv_bytes=157M gz_bytes=20.3M>,
 <LayoutMatch 'eric/noaa_gsod.global_summary_of_day@lat_lng_year' chunks=16 rows=924K csv_bytes=173M gz_bytes=22.2M>,
 <LayoutMatch 'eric/noaa_gsod.global_summary_of_day@year_month' chunks=111 rows=37.1M csv_bytes=6.62G gz_bytes=1.02G>]

Individual LayoutMatches can be also be described for easier reading:

In [20]:
# Here, matches[0] matches query to the 'lat_lng' layout; it requires 836K rows to fully match query.
matches[0].describe()

# LayoutMatch
Endpointer: `eric/noaa_gsod.global_summary_of_day@lat_lng`

## Stats
Matched: 838K rows (across 11 chunks)
Filesize: 20.3MB (157MB uncompressed)

## Matched Label Columns (2)
- lat: Point.Latitude(step=0.703125, head=-90.0)
- lng: Point.Longitude(step=1.40625, head=-180.0)

## Chunks (11)
- chunk|+030.234375|-090.000000.csv.gz (73806 rows)
- chunk|+034.453125|-087.187500.csv.gz (51113 rows)
- chunk|+040.078125|-075.937500.csv.gz (125631 rows)
- chunk|+040.078125|-074.531250.csv.gz (202167 rows)
- chunk|+040.781250|-075.937500.csv.gz (55106 rows)
...
- chunk|+041.484375|-080.156250.csv.gz (18635 rows)
- chunk|+041.484375|-077.343750.csv.gz (21046 rows)
- chunk|+042.187500|-080.156250.csv.gz (55822 rows)
- chunk|+042.187500|-074.531250.csv.gz (58261 rows)
- chunk|+042.890625|-077.343750.csv.gz (45515 rows)


In [21]:
# Here, matches[1] matches query to the 'lat_lng_year' layout; it requires 913K rows to fully match query.
matches[1].describe()

# LayoutMatch
Endpointer: `eric/noaa_gsod.global_summary_of_day@lat_lng_year`

## Stats
Matched: 924K rows (across 16 chunks)
Filesize: 22.2MB (173MB uncompressed)

## Matched Label Columns (3)
- lat: Point.Latitude(step=5.625, head=-90.0)
- lng: Point.Longitude(step=11.25, head=-180.0)
- year: Datetime.Year(step=1.0, head=1929.0)

## Chunks (16)
- chunk|+028.125000|-090.000000|2016.csv.gz (63383 rows)
- chunk|+033.750000|-090.000000|2016.csv.gz (80582 rows)
- chunk|+039.375000|-090.000000|2016.csv.gz (86275 rows)
- chunk|+039.375000|-078.750000|2012.csv.gz (73724 rows)
- chunk|+039.375000|-078.750000|2013.csv.gz (75778 rows)
...
- chunk|+039.375000|-078.750000|2020.csv.gz (77779 rows)
- chunk|+039.375000|-078.750000|2021.csv.gz (18248 rows)
- chunk|+039.375000|-033.750000|2016.csv.gz (649 rows)
- chunk|+039.375000|-033.750000|2017.csv.gz (706 rows)
- chunk|+039.375000|-033.750000|2018.csv.gz (713 rows)


In [22]:
# Here, matches[0] matches query to the 'year_month' layout; it requires 36.8M rows to fully match query!
matches[2].describe()

# LayoutMatch
Endpointer: `eric/noaa_gsod.global_summary_of_day@year_month`

## Stats
Matched: 37.1M rows (across 111 chunks)
Filesize: 1.02GB (6.62GB uncompressed)

## Matched Label Columns (2)
- year: Datetime.Year(step=1.0, head=1929.0)
- month: Datetime.Month(step=1.0, head=1.0)

## Chunks (111)
- chunk|2012|01.csv.gz (316283 rows)
- chunk|2012|02.csv.gz (298230 rows)
- chunk|2012|03.csv.gz (321361 rows)
- chunk|2012|04.csv.gz (314654 rows)
- chunk|2012|05.csv.gz (326946 rows)
...
- chunk|2020|11.csv.gz (337690 rows)
- chunk|2020|12.csv.gz (348044 rows)
- chunk|2021|01.csv.gz (343820 rows)
- chunk|2021|02.csv.gz (283377 rows)
- chunk|2021|03.csv.gz (336616 rows)


#### 4. Example: Fetching data using the `year_month` layout
While the gourdian library does its best to automatically choose the best layout for a Query, specific layouts can always be used to fine-tune the rows that are fetched as well.

Here we will use the `year_month` layout, which can be used to fetch data about a specific timeframe for all points on the globe:

In [23]:
match = local_df.gourdian.match(
    endpointer='eric/noaa_gsod.global_summary_of_day@year_month',
    how={
    'CRASH DATE': gtypes.Datetime.Year,
    'LATITUDE': gtypes.Point.Latitude,
    'LONGITUDE': gtypes.Point.Longitude,
    },
)
# For performance, paginate through the dataframes and render just the first one. The full dataframe is big!
dfs = match.dfs()
next(dfs).head()

GET 'https://data2.gourdian.net/files/datasets/eric/noaa_gsod/global_summary_of_day/year_month/chunks/chunk|2012|01.csv.gz'
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,71912099999,2012-01-01,56.35,-94.716667,145.0,"GILLAM AIRPORT, CA",8.2,24,3.5,24,...,20.0,25.1,21.2,*,-4.0,*,0.13,G,8.7,1000
1,71912099999,2012-01-02,56.35,-94.716667,145.0,"GILLAM AIRPORT, CA",-16.3,23,-25.5,23,...,20.0,21.0,-3.6,*,-22.0,*,0.01,G,7.9,0
2,71912099999,2012-01-03,56.35,-94.716667,145.0,"GILLAM AIRPORT, CA",1.6,24,-6.1,24,...,15.9,22.9,15.8,*,-13.5,*,0.00,G,8.3,1000
3,71912099999,2012-01-04,56.35,-94.716667,145.0,"GILLAM AIRPORT, CA",13.8,24,5.8,24,...,20.0,27.0,24.8,*,-4.0,*,0.00,G,7.5,1000
4,71912099999,2012-01-05,56.35,-94.716667,145.0,"GILLAM AIRPORT, CA",-5.6,24,-12.3,24,...,12.0,999.9,-2.2,*,-11.2,*,0.07,G,7.5,1000


#### 5. Example: Fetching data using the `lat_lng` layout

Here we will use the `lat_lng` layout, which can be used to fetch data about specific points on the globe for all timeframes:

In [24]:
match = local_df.gourdian.match(
    endpointer='eric/noaa_gsod.global_summary_of_day@year_month',
    how={
    'CRASH DATE': gtypes.Datetime.Year,
    'LATITUDE': gtypes.Point.Latitude,
    'LONGITUDE': gtypes.Point.Longitude,
    },
)
# Just show information about this match for now; data can be retrieved with df() as usual.
match.fetch_all_stats()

{'matched_num_chunks': 111,
 'matched_num_rows': 37106676,
 'matched_csv_bytes': 6619436945,
 'matched_gz_bytes': 1019511809}